# スクレイピングプログラム：YahooNewsのTopicsを入手

In [15]:
# データ入力用
from bs4 import BeautifulSoup as bs
import requests
import urllib
import time
import openpyxl

# データ加工用
import pandas as pd

# Yahooニュース一覧より
main_url = 'https://news.yahoo.co.jp/'
categories = ['', 'domestic', 'world', 'business', 'entertainment', 'sports', 'it', 'science', 'local']
dict = {}


# カテゴリ別のURL入手
def make_url(string):
    if string:
        return main_url + 'categories/' + string
    else:
        return main_url

    
for category in categories:
    url = make_url(category)
    html = requests.get(url)
    soup = bs(html.content, 'html.parser')
    time.sleep(1)#サーバ負荷を減らすため一秒待機

    title = []
    links = []

    topics = soup.find(id='uamods-topics')

    for element in topics.find_all('a'):

        if element.text == 'もっと見る':
            break
            
        title.append(element.text)
        
        href = element.get('href')
        link = urllib.parse.urljoin(url, href)
        links.append(link)

    if category == '':
        category = 'main'
    dict[category] = title
    dict[category + ':LINK'] = links
    
df = pd.DataFrame(dict)

## データの表示とダウンロード

In [28]:
while True:
    while True:
        choice = input('Select Category:')
        if choice in df or choice == 'all':
            print('\n')
            break
        print(choice, 'is not available')
        print('Please select from...', 'main', *categories[1:], 'all', sep='  ')

    if choice == 'all':
        selected_df = df
    else:
        selected_df = df[[choice, choice+':LINK']]
    print(selected_df, '\n')

    print('Download?','0:No', '1:Yes(csv)', '2:Yes(excel)')
    select_download = input()
    if select_download == '1':
        filename = 'download/YahooNews_' + choice + '.csv'
        selected_df.to_csv(filename, index=False)
        print('complete:', filename)
    elif select_download == '2':
        filename = 'download/YahooNews_' + choice + '.xlsx'        
        with pd.ExcelWriter(filename) as writer:
            selected_df.to_excel(writer, sheet_name=choice, index=False)
        print('complete:', filename)

    print('continue?', '0:No', '1:Yes')
    select_continue = input()
    if select_continue == '1':
        continue
        
    break

Select Category:all


               main                                main:LINK  \
0   宣言解除は五輪ありき 批判の声  https://news.yahoo.co.jp/pickup/6396305   
1   東京解除に否定的意見も 分科会  https://news.yahoo.co.jp/pickup/6396306   
2   ネットカフェで立てこもり 埼玉  https://news.yahoo.co.jp/pickup/6396308   
3   深夜に銀座 議員の自民復党浮上  https://news.yahoo.co.jp/pickup/6396309   
4   感染者らの郵便投票 仕組み解説  https://news.yahoo.co.jp/pickup/6396301   
5  65年前に小6が手紙 本人名乗り  https://news.yahoo.co.jp/pickup/6396304   
6   広島会沢 故障で五輪代表を辞退  https://news.yahoo.co.jp/pickup/6396310   
7   さらば森田 愛しい事務所の会長  https://news.yahoo.co.jp/pickup/6396303   

             domestic                            domestic:LINK  \
0     宣言解除は五輪ありき 批判の声  https://news.yahoo.co.jp/pickup/6396305   
1     東京解除に否定的意見も 分科会  https://news.yahoo.co.jp/pickup/6396306   
2     首相 医療ひっ迫なら酒提供停止  https://news.yahoo.co.jp/pickup/6396296   
3     緊急宣言 9都道府県の解除決定  https://news.yahoo.co.jp/pickup/6396279   
4  10都道府県 まん延防止7/11まで  https://news.yahoo.co.jp/pickup/6396283   
5    